In [2]:
import json
import requests
import re
import random
import os

In [12]:
def find_url_by_pattern(page_text, pattern):
    match = re.search(pattern, page_text)
    return match.group(0) if match else None

def compile_regex_for_action(action_type):
    # base_url_pattern = f"https://hearthstone\.wiki\.gg.*?_{action_type}_[0-9][0-9]\.wav"
    base_url_pattern = ''
    return re.search(base_url_pattern, re.IGNORECASE)


def find_urls_from_page(page_text):
    action_types = ['Play', 'Attack', 'Death']
    results = {}
    next_action = None
    search_pattern = '<source\s+src="([^"]+\.wav(?:\?[^"]*)?)"'
    full_art = None
    starter="<source src="
    for line in page_text:
        if not full_art:
            # full_art_temp = re.search('/wiki/File:[^"]*\.jpg', line)
            full_art_temp = re.search('\/images\/.*?_full.jpg', line)
            full_art = full_art_temp.group(0) if full_art_temp else None
        if next_action:
            result = find_url_by_pattern(line, search_pattern)
            if result:
                results[next_action] = result[len(starter) + 1:-1]
            next_action = None
        for action in action_types:
            header = f'<dl><dt>{action}</dt></dl>'
            if line.strip() == header:
                next_action = action
                break
        
            # pattern = compile_regex_for_action(action)
            # url = find_url_by_pattern(page_text, pattern)
            # if url:
            #     results[action.lower()] = url
                

    return (results if len(results) == len(action_types) else [], full_art)
file_path = "../JsonFiles/final_cards.json"

In [10]:
#Retrieves data
all_cards = json.loads(open(file_path).read())
missed_cards = []
not_enough_cards = []
voicelines = []
for i,key in enumerate(all_cards):
	card_data = all_cards[key]
	if card_data["cardType"] == "Minion":
		card_name = card_data["name"]
		card_name = card_name.replace(" ", "_").replace('’', "'")
		save_path = f"../HTMLPages/{card_name}.html"
		card_page_url = f"https://hearthstone.wiki.gg/wiki/{card_name}"
		if os.path.exists(save_path):
			continue
		request = requests.get(card_page_url)
		if request.status_code != 200:
			missed_cards.append(key)
		else:
			out = open(save_path, "w")
			out.write(request.text)
			continue
	if i % 500 == 0:
		print(i)

In [22]:
all_cards = json.loads(open(file_path, "r").read())
missed_cards = []
voicelines = []
for i,key in enumerate(all_cards):
	card_data = all_cards[key]
	if card_data["cardType"] == "Minion":
		card_name = card_data["name"]
		card_name = card_name.replace(" ", "_").replace('’', "'")
		# try:
		page_text = open(f"../HTMLPages/{card_name}.html").readlines()
		(voicelines, full_art) = find_urls_from_page(page_text)
		if full_art:
			full_art = full_art[:8] + full_art[8+6:]
			full_art_url = f"https://hearthstone.wiki.gg{full_art}"
		# print(voicelines)
		card_data["voicelines"] = voicelines
		card_data["fullArt"] = full_art_url
		# except:
		# 	pass
	if i % 500 == 0:
		print(i)
for card_name in list(all_cards.keys()):
	if len(all_cards[card_name]['voicelines']) == 0:
		print(card_name)
		del all_cards[card_name]
outfile = open(file_path, "w")
outfile.write(json.dumps(all_cards, indent=4))

0
500
1000
1500
2000
2500
3000


3047729

In [12]:
# file_count = 3
# with open(file_path, "r") as all_cards:
# 	card_names = list(json.loads(all_cards.read()).keys())
# 	for i in range(file_count):
# 		with open(f"../{i}-list", "w") as outfile:
# 			random.shuffle(card_names)
# 			outfile.write("\n".join(card_names))

